In [1]:
import carla
import random
import os
from queue import Queue
from queue import Empty

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

In [2]:

# def sensor_callback(sensor_data, sensor_queue, sensor_name):
#     if 'lidar' in sensor_name:
#         sensor_data.save_to_disk(os.path.join('../outputs/output_synchronized', '%06d.ply' % sensor_data.frame))
#     if 'camera' in sensor_name:
#         sensor_data.save_to_disk(os.path.join('../outputs/output_synchronized', '%06d.png' % sensor_data.frame))
#     sensor_queue.put((sensor_data.frame, sensor_name))


actor_list = []
sensor_list = []
try:
    # First of all, we need to create the client that will send the requests, assume port is 2000
    

    # Retrieve the world that is currently running
    world = client.get_world()
    # world = client.load_world('Town02') # you can also retrive another world by specifically defining
    blueprint_library = world.get_blueprint_library()
    # Set weather for your world
    weather = carla.WeatherParameters(cloudiness=10.0,
                                        precipitation=10.0,
                                        fog_density=10.0)
    world.set_weather(weather)

    # set synchorinized mode
    original_settings = world.get_settings()
    settings = world.get_settings()
    settings.fixed_delta_seconds = 0.1
    settings.synchronous_mode = True
    world.apply_settings(settings)

    traffic_manager = client.get_trafficmanager()
    traffic_manager.set_synchronous_mode(True)

    # create sensor queue
    sensor_queue = Queue()

    # create the ego vehicle
    ego_vehicle_bp = blueprint_library.find('vehicle.audi.a2')
    # black color
    ego_vehicle_bp.set_attribute('color', '0, 0, 0')
    # get a random valid occupation in the world
    transform = random.choice(world.get_map().get_spawn_points())
    # spawn the vehilce
    ego_vehicle = world.spawn_actor(ego_vehicle_bp, transform)
    # set the vehicle autopilot mode
    ego_vehicle.set_autopilot(True)
    # collect all actors to destroy when we quit the script
    actor_list.append(ego_vehicle)

    # create directory for outputs
    # output_path = '../outputs/output_synchronized'
    # if not os.path.exists(output_path):
    #     os.makedirs(output_path)

    # add a camera
    # camera_bp = blueprint_library.find('sensor.camera.rgb')
    # camera relative position related to the vehicle
    # camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
    # camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego_vehicle)
    # set the callback function
    # camera.listen(lambda image: sensor_callback(image, sensor_queue, "camera"))
    # sensor_list.append(camera)

    # we also add a lidar on it
    # lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
    # lidar_bp.set_attribute('channels', str(32))
    # lidar_bp.set_attribute('points_per_second', str(90000))
    # lidar_bp.set_attribute('rotation_frequency', str(40))
    # lidar_bp.set_attribute('range', str(20))

    # set the relative location
    # lidar_location = carla.Location(0, 0, 2)
    # lidar_rotation = carla.Rotation(0, 0, 0)
    # lidar_transform = carla.Transform(lidar_location, lidar_rotation)
    # spawn the lidar
    # lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=ego_vehicle)
    # lidar.listen(
    #     lambda point_cloud: sensor_callback(point_cloud, sensor_queue, "lidar"))
    # sensor_list.append(lidar)

    while True:
        world.tick()
        # set the sectator to follow the ego vehicle
        spectator = world.get_spectator()
        transform = ego_vehicle.get_transform()
        spectator.set_transform(carla.Transform(transform.location + carla.Location(z=20),
                                                carla.Rotation(pitch=-90)))

        # As the queue is blocking, we will wait in the queue.get() methods
        # until all the information is processed and we continue with the next frame.
        # try:
        #     for i in range(0, len(sensor_list)):
                # s_frame = sensor_queue.get(True, 1.0)
                # print("    Frame: %d   Sensor: %s" % (s_frame[0], s_frame[1]))

        # except Empty:
        #     print("   Some of the sensor information is missed")

finally:
    world.apply_settings(original_settings)
    print('destroying actors')
    client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])
    for sensor in sensor_list:
        sensor.destroy()
    print('done.')

destroying actors
done.


KeyboardInterrupt: 

In [3]:
client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])